In [ ]:
import os
from os.path import join
from typing import List

import sys
sys.path.insert(0,'..')
from src.test_data import TEST_EXAMPLES


from ipywidgets import interact, Checkbox
from tokenizers import Tokenizer
from transformers import PreTrainedTokenizerFast

from plots import plot_histogram, compare_vocab, plot_overview, plot_timelines, plot_overview_data, plot_vocab_size


import numpy as np
import seaborn as sns
from itertools import product

from termcolor import colored

In [ ]:
OUTPUT_DIR = "../output"

In [ ]:
def get_models() -> List[str]:
    return [elem for elem in sorted(os.listdir(OUTPUT_DIR)) if not elem.startswith(".")]
    
models = get_models()
models

# 1. Show examples

In [ ]:
"\N{ANGSTROM SIGN}", "\N{LATIN CAPITAL LETTER A WITH RING ABOVE}", "\u0041\u030A"

In [ ]:
test_examples = TEST_EXAMPLES + [
    'Allmänna Allmänna',
    "<|endoftext|> test"
]

In [ ]:
def show_example_model(example, model, show_tokenization):
    _id = model.split("_")[0]
    tokenizer_file = join(OUTPUT_DIR, model, "tokenizer.json")
    tokenizer = Tokenizer.from_file(tokenizer_file)
    tokenizer_fast = PreTrainedTokenizerFast(tokenizer_file=tokenizer_file)
    encoding = tokenizer_fast.encode(example)
    print(f"============ {model}")
    print(f"example: '{example}'")
    print(f"pre-tok: {tokenizer.pre_tokenizer.pre_tokenize_str(example)}")
    # print(encoding)
    example_encoded = tokenizer_fast.convert_ids_to_tokens(encoding)
    print(f"encoded: {example_encoded} --- {len(example_encoded)}")
    example_decoded = tokenizer_fast.decode(encoding)
    print(f"decoded: '{example_decoded}'")
    example_decoded_bytes = example_decoded.encode("utf-8")
    print(f"decoded as bytes: {example_decoded_bytes}")
    print()
    example_decoded_per_token = [tokenizer_fast.decode(elem).replace("\n", "↩\n").replace(" ", "-") for elem in encoding]
    
    if show_tokenization:
        COLORS = ["red", "blue"] # "green"] # , "blue", "magenta", "cyan"]
        for i, elem in enumerate(example_decoded_per_token):
            print(colored(elem, COLORS[i%len(COLORS)]), end="")
        print()
        print(f"> {len(example_decoded_per_token)} tokens")
        print()

In [ ]:
@interact
def show_examples(example=test_examples, model=["ALL"] + models, show_tokenization=False):
    if model == "ALL":
        for model in sorted(models):
            show_example_model(example, model, show_tokenization)
    else:
        show_example_model(example, model, show_tokenization)
        
    

In [ ]:
# STOP

In [ ]:
# example
"ℌej Hej --- TVÅ TVÅ TVÅ".encode("utf-8")  # ℌ, H --- ÅNGSTRÖM, Å, A+°

In [ ]:
# NFC
"ℌej Hej --- TVÅ TVÅ TVÅ".encode("utf-8")  # ℌ, H --- Å, Å, Å

In [ ]:
# NFKD
"Hej Hej --- TVÅ TVÅ TVÅ".encode("utf-8")  # H, H --- A+°, A+°, A+°

In [ ]:
# NFKC
"Hej Hej --- TVÅ TVÅ TVÅ".encode("utf-8")  # H, H --- Å, Å, Å

# 2. Subwords

### 2a. Subword Length Histograms

In [ ]:
@interact
def show_histogram(model_1=models, model_2=[None] + models, xlim=20, ylim=15000):
    plot_histogram(model_1, model_2, xlim, ylim)

### 2b. Overlap

In [ ]:
@interact
def show_compare_vocab(model_1=models, model_2=models, nr=5):
    v, ex1, ex2 = compare_vocab(model_1, model_2, 1000000, 1000000)
    print(v)
    print()
    print("=== only model 1 ===")
    print(ex1[:nr])
    print()
    print("=== only model 2 ===")
    print(ex2[:nr])

### 2c. Vocabulary Size & Subword Length Mean

In [ ]:
@interact
def show_vocab_size(model=models):
    plot_vocab_size(model)

# 3. Multilinguality

In [ ]:
models_multilinguality = [model for model in models if model.count("_3") > 0]
core = list(set(["_".join(model.split("_")[1:-1]) for model in models_multilinguality if model.endswith("da")]))[0]
print(core)
models_multilinguality = [model for model in models_multilinguality if core in model]
models_multilinguality.sort(key = lambda x: x.split("_3")[-1])
models_multilinguality = {model.split("_3")[-1]: model for model in models_multilinguality}
models_multilinguality

In [ ]:
lang = list(models_multilinguality.keys())
lang_all = [l for l in lang if l.startswith("all")]
lang_pure = [l for l in lang if not l.startswith("all")]

models = {k: models_multilinguality[k] for k in lang}
models_all = {k: models_multilinguality[k] for k in lang_all}
models_pure = {k: models_multilinguality[k] for k in lang_pure}

lang, lang_all, lang_pure

In [ ]:
models_pure

In [ ]:
# overview_corpus(models_multilinguality)

### 3a. Time

In [ ]:
plot_overview_data(models_pure.values())

In [ ]:
plot_overview(models_pure.values())

### 3b. Intersection Matrix (Subword Length)

In [ ]:
# get_intersection_matrix()

In [ ]:
# get_intersection_matrix(0)

In [ ]:
# get_intersection_matrix(10)

In [ ]:
# get_intersection_matrix(10000)

### 3c. Intersection Timeline (Subword Length)

In [ ]:
def get_intersection(lang_1, lang_2, vocab_1, vocab_2):
    model_1 = models_multilinguality[lang_1]
    model_2 = models_multilinguality[lang_2]
    v, _, _ = compare_vocab(model_1, model_2, vocab_1, vocab_2)
    return v["intersection"]

In [ ]:
get_intersection('all-a1.0', 'da', 10000, 10000)

In [ ]:
VOCAB = [10000, 20000, 30000, 40000, 50000, 100000, 150000, 200000, 250000]
VOCAB_1 = VOCAB
VOCAB_2 = VOCAB

# VOCAB_1 = [50000, 100000, 150000, 200000, 250000]
# VOCAB_2 = [100, 1000, 10000, 20000, 30000, 40000, 50000]

In [ ]:
intersections = {
    lang_1: {
        lang_2: {
            vocab_1: {
                vocab_2: get_intersection(lang_1, lang_2, vocab_1, vocab_2)
                for vocab_2 in VOCAB_2
            }
            for vocab_1 in VOCAB_1
        }
        for lang_2 in lang
    }
    for lang_1 in lang_all
}

# intersections

In [ ]:
timelines_abs = {
    lang_1: {
        vocab_2: {
            lang_2: 
            [intersections[lang_1][lang_2][vocab_1][vocab_2] for vocab_1 in VOCAB_1]
            for lang_2 in lang_pure
        }
        for vocab_2 in VOCAB_2
    }
    for lang_1 in lang_all
}
# timelines_abs

In [ ]:
timelines_rel = {
    lang_1: {
        vocab_2: {
            lang_2: 
            [intersections[lang_1][lang_2][vocab_1][vocab_2]/intersections[lang_1][lang_1][vocab_2][vocab_2] for vocab_1 in VOCAB_1]
            for lang_2 in lang_pure
        }
        for vocab_2 in VOCAB_2
    }
    for lang_1 in lang_all
}
# timelines_rel

In [ ]:
%%javascript
IPython.OutputArea.prototype._should_scroll = function(lines) {
    return false;
}

In [ ]:
@interact
def show_timelines(tokenizer=lang_all, vocab_size=VOCAB_2):
    lang_1 = tokenizer
    vocab_2 = vocab_size
    t_abs = timelines_abs[lang_1][vocab_2]
    t_rel = timelines_rel[lang_1][vocab_2]
    
    plot_timelines(
        VOCAB_1,
        vocab_2,
        [t_abs, t_rel],
        lang_pure, 
        ylim=[1.1*100000, 1.1],
        ylabel=["absolute", "relative"], 
        title=["Coverage of single-language tokenizer vocabulary"]*2,
    )